In [ ]:
from datetime import datetime  
from functools import partial
import numpy as np
import tensorflow as tf
import importlib
from tensorflow.keras import regularizers
from tensorflow.python.ops.numpy_ops import np_config
import io_functions as io
import plotting_functions as plf
np_config.enable_numpy_behavior()

## Data Preparation

### Description
The data used for training the model was generated in numerical simulations in the .root format. It was then converted to the .tfrecord format using the `ROOT_to_TFRecord.ipynb` notebook and the `conversion_xyz` function.

This notebook presents the xyz reconstruction approach, where the model takes all three projections in uvwt coordinates and performs reconstruction to obtain the xyz coordinates of an event.

In [ ]:
dataPath = "/scratch/pszyc"
batchSize = 32

In [ ]:
filenames = [f"{dataPath}/data_xyz/out_random_sigma-001-part-{i}.tfrecord" for i in range(5)]
train_dataset = tf.data.TFRecordDataset(filenames, compression_type='GZIP', num_parallel_reads=5)
test_dataset = tf.data.TFRecordDataset(f"{dataPath}/data_xyz/out_random_sigma2k2mm-part-0.tfrecord", compression_type='GZIP')
# Create a description of the features.
feature_description = {
    'myChargeArray': tf.io.FixedLenFeature([], tf.string),
    'target': tf.io.FixedLenFeature([], tf.string),

}

def _parse_function(example_proto):
  # Parse the input `tf.train.Example` proto using the dictionary above.
    parsed_features = tf.io.parse_single_example(example_proto, feature_description)
    charge, target = parsed_features['myChargeArray'], parsed_features['target']
    # decode from bytes
    charge = tf.io.parse_tensor(charge, tf.float64)
    target = tf.io.parse_tensor(target, tf.float64)
    
    return charge, target

train_dataset = train_dataset.map(_parse_function, num_parallel_calls=tf.data.AUTOTUNE)
test_dataset = test_dataset.map(_parse_function, num_parallel_calls=tf.data.AUTOTUNE)

train_dataset = train_dataset.unbatch()
test_dataset = test_dataset.unbatch()

train_dataset = train_dataset.batch(batchSize)
test_dataset = test_dataset.batch(batchSize)

for aBatch in train_dataset.take(1):
    plf.plotEvent(aBatch, model=None)
    pass

for aBatch in test_dataset.take(1):
    plf.plotEvent(aBatch, model=None)
    pass

# Training

In [ ]:
regularizer = regularizers.l2(0.01)
model = tf.keras.Sequential([
  tf.keras.layers.InputLayer(input_shape = (256, 512, 3)), 
    tf.keras.layers.Conv2D(16, 5, padding='same', activation='relu', data_format="channels_last", input_shape=(256, 512, 1), kernel_regularizer=regularizer),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Conv2D(32, 5, padding='same', activation='relu', data_format="channels_last", kernel_regularizer=regularizer),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Conv2D(64, 5, padding='same', activation='relu', data_format="channels_last", kernel_regularizer=regularizer),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu', kernel_regularizer=regularizer),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(64, activation='relu', kernel_regularizer=regularizer),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(32, activation='relu', kernel_regularizer=regularizer),
    tf.keras.layers.Dense(9)
])

initial_learning_rate = 0.001
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(initial_learning_rate,
                decay_steps=5000,
                decay_rate=0.98,
                staircase=False)

optimizer = tf.keras.optimizers.RMSprop(learning_rate=lr_schedule) 
model.compile(optimizer = optimizer, 
              loss = 'mse', 
              metrics=['mse'])

In [ ]:
log_dir = "logs/fit/" + datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1, profile_batch=(10, 20))
early_stop_callback = tf.keras.callbacks.EarlyStopping(patience=5, verbose=1)
callbacks =  [early_stop_callback, tensorboard_callback]

In [ ]:
epochs=5
history = model.fit(train_dataset, 
                    epochs=epochs,
                    workers = 3,
                    use_multiprocessing = True,
                    validation_data = test_dataset.take(10),
                    callbacks=callbacks
                    )

In [ ]:
current_time = datetime.now().strftime("%Y_%b_%d_%H_%M_%S")

job_dir = f"{dataPath}/{epochs:04d}_"+current_time
model.save(job_dir.format(epochs=epochs), save_format='tf')

## Model performance

Fill Pandas DataFrame with true and response values.

In [ ]:
import utility_functions as utils
importlib.reload(utils)

current_time = datetime.now().strftime("%Y_%b_%d_%H_%M_%S")
print("Inference. Current Time =", current_time)

#model_path = "./training/2023_Apr_28_16_58_32/"
#model_path = "/scratch_hdd/akalinow/ELITPC/PythonAnalysis/training/2023_May_02_23_51_30/"
#model = tf.keras.models.load_model(model_path)

df = utils.df

for aBatch in test_dataset: 
    df = utils.fillPandasDataset(aBatch, df, model)     
    
for aBatch in test_dataset.take(5):
    plf.plotEvent(aBatch, model=model)

df.describe()    

### Resolution plots

In [ ]:
import plotting_functions as plf
importlib.reload(plf)

#plf.controlPlots(df)
plf.plotEndPointRes(df=df, edge="Start", partIdx=1)
plf.plotEndPointRes(df=df, edge="Stop", partIdx=1)

plf.plotEndPointRes(df=df, edge="Start", partIdx=2)
plf.plotEndPointRes(df=df, edge="Stop", partIdx=2)

plf.plotLengthPull(df, partIdx=1)
plf.plotLengthPull(df, partIdx=2)
plf.plotLengthPullEvolution(df)
plf.plotOpeningAngleCos(df)